# Finding Anomalies for an Entire Era
This notebook is for compiling an Excel file detailing all of the anomalies in a single era. 

Specify the Era

1. Load the appropriate model
2. Import all runs/lumisections that pass the DCS flags
3. Predict on the lumisections
4. Loop over each run
    * Split the long combined lumisection, data, and predictions arrays with their respective run number
    * Store the predictions with the specific run in a dictionary
    * Keep a running list of each dictionary
5. Loop over each run
    * Calculate the losses and binary losses (Do this in separate loop so we can change the loss threshold)
6. Loop over each run and analyze the anomalies
    * Try to figure out a data storage format that we can run through the normal Excel creation format
7. Create an plot for each anomalous lumisection

In [13]:
%%time
# imports

import os
import sys
import json
import time
import joblib
import importlib
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import plottools as plottools

from nmf2d import NMF2D

import functions
importlib.reload(functions);
from functions import *
    
#Used to load an extension that can skip the remaining execution of a cell. 
#Used to skip the training so we don't constantly retrain a model
%load_ext skip_kernel_extension
%reload_ext skip_kernel_extension

optimized_powerGroupStringsList = np.array(['FPix_BmO_D3_ROG4','FPix_BmO_D2_ROG4','FPix_BmO_D1_ROG4','FPix_BmO_D3_ROG3','FPix_BmO_D2_ROG3','FPix_BmO_D1_ROG3','FPix_BmO_D3_ROG2','FPix_BmO_D2_ROG2','FPix_BmO_D1_ROG2','FPix_BmO_D3_ROG1','FPix_BmO_D2_ROG1','FPix_BmO_D1_ROG1','FPix_BmI_D3_ROG1','FPix_BmI_D2_ROG1','FPix_BmI_D1_ROG1','FPix_BmI_D3_ROG2','FPix_BmI_D2_ROG2','FPix_BmI_D1_ROG2','FPix_BmI_D3_ROG3','FPix_BmI_D2_ROG3','FPix_BmI_D1_ROG3','FPix_BmI_D3_ROG4','FPix_BmI_D2_ROG4','FPix_BmI_D1_ROG4','FPix_BpO_D1_ROG4','FPix_BpO_D2_ROG4','FPix_BpO_D3_ROG4','FPix_BpO_D1_ROG3','FPix_BpO_D2_ROG3','FPix_BpO_D3_ROG3','FPix_BpO_D1_ROG2','FPix_BpO_D2_ROG2','FPix_BpO_D3_ROG2','FPix_BpO_D1_ROG1','FPix_BpO_D2_ROG1','FPix_BpO_D3_ROG1','FPix_BpI_D1_ROG1','FPix_BpI_D2_ROG1','FPix_BpI_D3_ROG1','FPix_BpI_D1_ROG2','FPix_BpI_D2_ROG2','FPix_BpI_D3_ROG2','FPix_BpI_D1_ROG3','FPix_BpI_D2_ROG3','FPix_BpI_D3_ROG3','FPix_BpI_D1_ROG4','FPix_BpI_D2_ROG4','FPix_BpI_D3_ROG4'])
#A list of all of the quarters of the detector
QUARTERS = np.array([['FPix_BmI_D3_ROG1','FPix_BmI_D3_ROG2','FPix_BmI_D3_ROG3','FPix_BmI_D3_ROG4','FPix_BmI_D2_ROG1','FPix_BmI_D2_ROG2','FPix_BmI_D2_ROG3','FPix_BmI_D2_ROG4','FPix_BmI_D1_ROG1','FPix_BmI_D1_ROG2','FPix_BmI_D1_ROG3','FPix_BmI_D1_ROG4'], ['FPix_BmO_D3_ROG1','FPix_BmO_D3_ROG2','FPix_BmO_D3_ROG3','FPix_BmO_D3_ROG4','FPix_BmO_D2_ROG1','FPix_BmO_D2_ROG2','FPix_BmO_D2_ROG3','FPix_BmO_D2_ROG4','FPix_BmO_D1_ROG1','FPix_BmO_D1_ROG2','FPix_BmO_D1_ROG3','FPix_BmO_D1_ROG4'], ['FPix_BpI_D1_ROG1','FPix_BpI_D1_ROG2','FPix_BpI_D1_ROG3','FPix_BpI_D1_ROG4','FPix_BpI_D2_ROG1','FPix_BpI_D2_ROG2','FPix_BpI_D2_ROG3','FPix_BpI_D2_ROG4','FPix_BpI_D3_ROG1','FPix_BpI_D3_ROG2','FPix_BpI_D3_ROG3','FPix_BpI_D3_ROG4'], ['FPix_BpO_D1_ROG1','FPix_BpO_D1_ROG2','FPix_BpO_D1_ROG3','FPix_BpO_D1_ROG4','FPix_BpO_D2_ROG1','FPix_BpO_D2_ROG2','FPix_BpO_D2_ROG3','FPix_BpO_D2_ROG4','FPix_BpO_D3_ROG1','FPix_BpO_D3_ROG2','FPix_BpO_D3_ROG3','FPix_BpO_D3_ROG4']])

The skip_kernel_extension extension is already loaded. To reload it, use:
  %reload_ext skip_kernel_extension
CPU times: user 9.22 ms, sys: 4.57 ms, total: 13.8 ms
Wall time: 1.29 s


|        2024 Era     | C | D | E | E | F |   F  |   G  |   H  |   I  |   I  |
|:-------------------:|:-:|:-:|:-:|:-:|:-:|:----:|:----:|:----:|:----:|:----:|
|       Version       | 1 | 1 | 1 | 2 | 1 |   1  |   1  |   1  |   1  |   2  |
|        Period       | 1 | 1 | 1 | 1 | 1 |   2  |   2  |   2  |   2  |   2  |
|        Model        | 1 | 1 | 1 | 1 | 1 |   2  |   2  |   2  |   2  |   2  |

|        2025 Era     | C | C | C | D | E | F |   G  |   H  |   I  |   J  |
|:-------------------:|:-:|:-:|:-:|:-:|:-:|:-:|:----:|:----:|:----:|:----:|
|       Version       | 1 | 1 | 2 | 1 | 1 | 1 |   1  |   1  |   1  |   1  |
|        Period       | 3 | 4 | 4 | X | X | X |   X  |   X  |   X  |   X  |
|        Model        | 3 | 4 | 4 | X | X | X |   X  |   X  |   X  |   X  |

# Important Variables

In [2]:
#Select the era: import and search for anomalies
RING = 1
YEAR, ERA, VERSION, PERIOD = 2024, "C", 1, 1 

number, type = 1, 1
model_name = f'model_{number}_PXRing_{RING}_period_{PERIOD}_type_{type}.pkl'
#Folder --> /eos/user/a/alaperto/SWAN_projects/NMFolder/models/
#Data by Lyka --> /eos/user/l/llambrec/dialstools-output/
#Original Jake --> /eos/user/j/jomorris/SWAN_projects/NMF Testing/

# Model thresholds
EDITION = 1
if YEAR == 2024:
    LOSS_THRESHOLD = 4e5 #Threshold on Loss of the ROC --> Define ROC as Bad
elif YEAR == 2025:
    LOSS_THRESHOLD = 9e5
ANOMALY_CUTOFF = 40 #Threshold on fraction of powergroup that is Bad --> Define LS as anomalous

#Plotting Globals
DO_PLOTTING = False #Whether or not to plot EVERY anomalous lumisection in this era. WARNING: Takes a long time. ~13mins for 93 anomalies
SAVE_FIGS = False #Whether or not to also SAVE the plot of every anomalous lumisection. DO_PLOTTING must also be True for the images to be saved

#Some light calculation of important variables
file = f'../data/ZeroBias-Run{YEAR}{ERA}-PromptReco-v{VERSION}-DQMIO-PixelPhase1-Phase1_MechanicalView-PXForward-clusters_per_SignedDiskCoord_per_SignedBladePanelCoord_PXRing_{RING}.parquet'
oms_json = f'../omsdata/omsdata_Run{YEAR}{ERA}-v{VERSION}.json'
#ring_num = int(file[-9]) #The -9th character is ALWAYS the ring number for our data
ring_num = RING

#The directory name to use for 
DIR_NAME = f"../results/output_{YEAR}{ERA}_v{VERSION}_PXRing_{RING}_edition_{EDITION}" 
if not os.path.exists(DIR_NAME): os.makedirs(DIR_NAME)

## 1. Load the Model

In [3]:
nmf_file = f'../models/{model_name}'
nmf = joblib.load(nmf_file)

print(f"Loaded Model: {nmf_file}")
print(f"Model Shape: {nmf.xshape}")

Loaded Model: ../models/model_1_PXRing_1_period_1_type_1.pkl
Model Shape: [88, 48]


## 2. Import all runs/lumisections that pass the DCS flags
Import the entire era at once so we only have to go to the disk once. 

Use the OMS JSON to filter only the lumisections that pass the DCS flags. Helps to reduce unnecessary predictions on bad lumisections

In [4]:
%%time
#Any logic for when parts of the detector are disabled. 
extra_filters = []
#Logic for Era F period 1/2 and 2025 Era Cv1 period 3/4
if YEAR == 2024 and ERA == "F" and PERIOD == 1:
    extra_filters.append(('run_number', '<', 382799))
elif YEAR == 2024 and ERA == "F" and PERIOD == 2:
    extra_filters.append(('run_number', '>=', 382799))
elif YEAR == 2025 and ERA == "C" and VERSION == 1 and PERIOD == 3:
    extra_filters.append(('run_number', '<=', 392668))
elif YEAR == 2025 and ERA == "C" and VERSION == 1 and PERIOD == 4:
    extra_filters.append(('run_number', '>', 392668))
    extra_filters.append(('run_number', '<', 393512))#Machine Development runs

#Testing the function to import AND filter an entire era at once
multi_lumi_data, all_runs, lumis, df = extract_data_whole_era(file, oms_json, extra_filters=extra_filters)
del df
all_runs, indices = np.unique(all_runs, return_index=True)
indices = indices[1:] #The first index in indices is always 0, since the first number is always unique, so we discard that. 
print(f"There are {len(all_runs)} Runs and {len(lumis)} Lumisections that Pass All DCS Flags: \n", all_runs)

#Optional if you'd like to see how many TOTAL runs there are, as most runs are entirely disqualified due to DCS Flags. 
# dataset = ParquetDataset(file).read().to_pandas()
# all_runs = np.array(dataset["run_number"].unique())
# print(f"There are {len(all_runs)} Total Available Runs: \n", all_runs)

There are 42278 rows in the Parquet dataset but only 42277 rows in the merged dataframe! 
There are 1 lumisections that were thrown out due to no DCS flags present!

There are 53 Runs and 25159 Lumisections that Pass All DCS Flags: 
 [379415 379416 379420 379425 379433 379442 379454 379456 379470 379523
 379524 379530 379613 379614 379615 379616 379617 379618 379660 379661
 379729 379765 379774 379866 379984 380001 380005 380006 380007 380029
 380030 380032 380033 380043 380049 380050 380051 380052 380053 380056
 380066 380074 380115 380126 380127 380128 380195 380196 380197 380235
 380236 380237 380238]
CPU times: user 4.4 s, sys: 4.09 s, total: 8.49 s
Wall time: 10.1 s


## 3. Predict on the lumisections

In [5]:
%%time

# Predict or not predict (30 mins for 25k LS)
FORCE_PREDICT = False

#Remove the cross so we can predict on it
multi_lumi_data_no_cross = remove_cross(multi_lumi_data, RING)

#Save predictions array to a file (since they take so long to produce)
pred_filename = f"{DIR_NAME}/Predictions"

if os.path.exists(pred_filename + '.npy') and FORCE_PREDICT == False:
    print(f"Predictions Already Exist!")
    print(f"Loading predictions from {pred_filename}.npy")
    mes_pred = np.load(pred_filename + '.npy')
    print(f"Shape: {mes_pred.shape}")
else:
    #If we don't already have a prediction directory then make it
    if FORCE_PREDICT: 
        print(f"FORCE_PREDICT is True. ---> Overwrite file at {pred_filename}.npy")
    else:
        print("Predictions don't exist yet! Starting Prediction. ")

    #Predict on the data
    print(f'Predicting...')
    start_time = time.time()
    mes_pred = nmf.predict(multi_lumi_data_no_cross)
    np.save(pred_filename, mes_pred)
    print(f"Done Predicting in {time.time() - start_time} Seconds!\n")

Predictions Already Exist! Loading predictions from ../results/output_2024C_v1_PXRing_1_edition_1/Predictions
Predictions with (25159, 88, 48) shape loaded from ../results/output_2024C_v1_PXRing_1_edition_1/Predictions.npy
CPU times: user 278 ms, sys: 1.16 s, total: 1.44 s
Wall time: 9.2 s


## 4. Loop over each run
    * Split the long combined lumisection, data, and predictions arrays with their respective run number
    * Store the predictions with the specific run in a dictionary
    * Keep a running list of each dictionary

In [6]:
%%time
#Verbose=0: No prints. Verbose=1: Some prints. Verbose>=2: Some very long prints
verbose = 0

#Split the long combined arrays of the lumisections, the data, and the predictions
lumisections = np.split(lumis, indices)
data_arr = np.split(multi_lumi_data_no_cross, indices)
pred_arr = np.split(mes_pred, indices)

#print(pred_arr[1].shape)

#Now our data is splitted into arrays by run number, we can loop over the run numbers, generate our data_dicts and put them in a data_dict_list.
#So it is in a format ready to be used to calculate the losses and anomalies. 
#Create a list to store all of the lumisections and predictions
data_dict_list = list(np.empty_like(all_runs)) 
for index, run_number in enumerate(all_runs):
    data_dict = {}
    data_dict["run_number"] = run_number
    data_dict["lumisections"] = lumisections[index]
    data_dict["data"] = data_arr[index]
    data_dict["predictions"] = pred_arr[index]
    #Add the cross into the predictions and data
    data_dict["data_cross"] = add_cross(data_arr[index])
    data_dict["predictions_cross"] = add_cross(pred_arr[index])
    
    #Add this run to the data dict list
    data_dict_list[index] = data_dict    
    #Printing
    if verbose>0:
        print(f"Run Number: {run_number}")
        print(f"\tThere are {len(lumisections[index])} Extracted Lumisections:\n\t{lumisections[index]}\n")
    if verbose>1:
        print(data_dict)
        
if verbose>0:
    print(data_dict_list[0])
elif verbose>1:
    print(data_dict_list)

CPU times: user 1.89 s, sys: 195 ms, total: 2.08 s
Wall time: 2.08 s


## 5. Loop over each run
    * Calculate the losses and binary losses (Do this in separate loop so we can change the loss threshold)

In [7]:
%%time

#Loop over each data dictionary in the list and calculate the losses and binary losses. 
#Add these to the dictionaries as we go along. 
for index, data_dict in enumerate(data_dict_list):
    #Extract the needed info from the data_dict
    multi_lumi_data_no_cross = data_dict["data"]
    mes_pred = data_dict["predictions"]
    
    #Calculate losses
    losses = np.square(multi_lumi_data_no_cross - mes_pred)
    losses_binary = (losses > LOSS_THRESHOLD).astype(int)
    
    #Add the crosses back
    losses_cross = add_cross(losses)
    losses_binary_cross = add_cross(losses_binary)
    
    #Add new entries to the data_dict
    #This will automatically update the dictionaries in the data_dict_list
    data_dict["losses"] = losses_cross
    data_dict["losses_binary"] = losses_binary_cross

CPU times: user 2.21 s, sys: 567 ms, total: 2.78 s
Wall time: 2.78 s


## 6. Loop over each run and analyze the anomalies. 

### Analyze Each Lumisection of Each Run for Anomalies

In [8]:
%%time

testingtime = False
verbose = False

#Create lists for tracking the anomalous lumisections in all of the runs
all_anomalous_runs = []
all_anomalous_lumisections = []
all_anomalous_powergroups = []

for i, data_dict in enumerate(data_dict_list):
    #Extract the required info from the data_dict
    run_number = data_dict["run_number"]
    lumisections = data_dict["lumisections"]
    losses_binary_cross = data_dict["losses_binary"]
    
    #Currently a list of anomalous lumisections and their powergroups
    #These arrays are prevented from getting out of sync by still appending 
    #the anomalous lumisection even if that lumisection was already marked bad with a different powergroup
    anomalous_lumisections = []
    anomalous_powergroups = []

    for index, lumisection in enumerate(lumisections):
        if verbose: print(f"Index: {index} \t Lumisection: {lumisection}")

        for j, powergroup in enumerate(optimized_powerGroupStringsList):
            #if not testingtime: print(f"Power Group String: {powergroup}")
            powerGroupSlice, diskSlice = powerGroupToIndex(powergroup)

            #Access each power group in each lumisection and see if more than 50% of the bins are on
            #A bit ugly, but this was the fastest way I found. Saved about .1 seconds over saving the powergroup data to another variable. 
            if int(np.sum(losses_binary_cross[index, powerGroupSlice, diskSlice].flatten())) >= int(ANOMALY_CUTOFF/100 * losses_binary_cross[index, powerGroupSlice, diskSlice].flatten().size):
                if verbose: print(f"Anomalous Power Group: {powergroup} \t in Lumisection: {lumisection} \t with Binary Sum: {np.sum(powerGroup_data)}")
                all_anomalous_runs.append(run_number)
                anomalous_lumisections.append(lumisection)
                anomalous_powergroups.append(powergroup)
                
            #This is used to pull out specific lumisection and check their binary loss occupancy
#             if run_number == 381544 and lumisection == 1861:
#                 print(f"Powergroup: {powergroup}")
#                 print(f"Powergroup Size:{losses_binary_cross[index, powerGroupSlice, diskSlice].flatten().size}")
#                 print(f"Sum of Binary Loss:{np.sum(losses_binary_cross[index, powerGroupSlice, diskSlice].flatten())}\n")

                if verbose: save_digis_png(losses_binary_cross[index], testing_ring_num, lumisection, 1)
    #Update the data_dict with the anomalous lumisections and powergroups
    #I don't have a plan for them currently, but it could be useful
    data_dict["anomalous_lumisections"] = np.array(anomalous_lumisections)
    data_dict["anomalous_powergroups"] = np.array(anomalous_powergroups)
    
    #EXTEND the anomalous lumisections and powegroups to the ALL list. Extend keeps the list flat
    all_anomalous_lumisections.extend(anomalous_lumisections)
    all_anomalous_powergroups.extend(anomalous_powergroups)
    
# print(all_anomalous_runs)
# print(all_anomalous_lumisections)
# print(all_anomalous_powergroups)
# print(data_dict_list[-2])
num_anomalous_lumisections, all_anomalous_lumisections_unique = calcNumAnomalousLumisections(data_dict_list)
print(f'Resuming results...')
print(f"Era {YEAR}{ERA}_v{VERSION} (Run {all_runs[0]} to Run {all_runs[-1]})")
print(f"{num_anomalous_lumisections} Anomalous Lumisections")
#For some reason the above number can disagree with the sum of the Num_LS column in Excel. But they are never far off.
#Might happen due to lumisections appearing twice in the excel file. Like when a single lumisection has both a single disk anomaly and a multi disk anomaly
#The lumisection could be double counted. There may be more cases where this happens. 

Resuming results...
Era 2024C_v1 (Run 379415 to Run 380238)
92 Anomalous Lumisections
CPU times: user 22.2 s, sys: 507 µs, total: 22.2 s
Wall time: 22.2 s


## Identify Anomaly Types
This section will identify runs in lumisections that stay the same then compare the powergroups and see if there are any multi-disk anomalies. 

If there are no repeating lumisections, then it is just a single disk anomaly. 

### Create a file summarizing all lumisections and their anomalies
#### Run_Number    Lumisection    PRT    Disk    Ring_Num    Anomaly_Type

In [14]:
%%time
#Identify multi disk anomalies
verbose = 0
#Create a pandas dataframe that we can use to track EACH anomalous lumisection in each run
headers = ["Run_Number", "Lumisection", "Powergroup", "Disk", "Ring_Num", "Anomaly_Type"]
all_detailed_anomaly_df = pd.DataFrame(columns=headers)

#Loop over each data dict in the data dict list
for index, data_dict in enumerate(data_dict_list):
    #Extract the relavant information
    run_number = data_dict["run_number"]
    anomalous_lumisections = data_dict["anomalous_lumisections"]
    anomalous_powergroups = data_dict["anomalous_powergroups"]
    
    #If there are no anomaous lumisections or powergroups then stop this iteration
    if anomalous_lumisections.size == 0 or anomalous_powergroups.size == 0:
        continue
    

    #Create dataframe of anomalous lumisections and powergroups
    anomaly_df = pd.DataFrame({"lumisections": anomalous_lumisections, "powergroups": anomalous_powergroups})

    if verbose>0: print("Anomaly Dataframe: \n", anomaly_df, '\n')

    #Create unique arrays to pare down duplicate data
    anomalous_lumisections_unique = np.unique(anomalous_lumisections)

    #Create a list of dictionaries for easier saving to text
    dictList = np.empty_like(anomalous_lumisections_unique, dtype=dict)

    detailed_anomaly_df = pd.DataFrame(columns=headers)

    # print("AHHHHHHH\n", detailed_anomaly_df)
    if verbose>0: print('-----------------------------------------')
    #Loop over each unique lumisection
    for index, lumisection in enumerate(anomalous_lumisections_unique):
        #These values are the same for single/multi disk anomalies
        dataDict = dict.fromkeys(headers)
        dataDict["Run_Number"] = run_number
        dataDict["Lumisection"] = lumisection
        dataDict["Ring_Num"] = ring_num

        #Get the lumisection and all of the anomaly powergroups
        #If there is only one powergroup, mark it Single Disk, preparing the detailed Pandas anomaly dataframe, then move on
        #If there is multiple powergroups, iterate through each pair, breaking on the first Multi-disk anomaly after preparing the detailed Pandas anomaly dataframe
        #If there is no Multi-disk anomaly despite there being multiple anomalies in one lumisection, prepare the detailed Pandas anomaly dataframe with EACH anomaly


        dataframe = anomaly_df[anomaly_df["lumisections"] == lumisection]
        if verbose>0: print(dataframe, '\n')
        powergroups = dataframe["powergroups"].to_list()
        if verbose>0: print(f"Powergroups: {powergroups}")

        #If there are 12 anomalous powergroups in one lumisection, check if it's a whole quarter out
        if len(powergroups) == 12:
            for quarter in QUARTERS:
                #If all of the powergroups are in one quarter, then we can save and break early
                if np.all(np.isin(powergroups, quarter)):
                    #Fill in the data dict
                    m_or_p, I_or_O, disk_number, part_number = analyzePowerGroupString(powergroups[0])
                    dataDict["Powergroup"] = ':'.join(powergroups) #Make a string of each powergroup separated by colons. A char not typically used in csv's. 
                    dataDict["Disk"] = "-1:-2:-3" if disk_number < 0 else "1:2:3"
                    #dataDict["Anomaly_Type"] = "Whole Quarter"
                    dataDict["Anomaly_Type"] = "Multi-Disk"
                    dataFrame = pd.Series(dataDict).to_frame().T
                    #print(dataFrame)
                    detailed_anomaly_df = pd.concat([detailed_anomaly_df, dataFrame])
                    #break out of this for loop for the quarters
                    break
            #then continue to the next unique anomaly
            continue

        #If there is only one powergroup, mark it as a Single disk anomaly
        if len(powergroups) == 1:
            #Fill in the data dict
            m_or_p, I_or_O, disk_number, part_number = analyzePowerGroupString(powergroups[0])
            dataDict["Powergroup"] = powergroups[0]
            dataDict["Disk"] = disk_number
            dataDict["Anomaly_Type"] = "Single-Disk"
            #Convert the data dict to a pandas dataframe and concat it to the detailed data frame 
            #(NOTE: If there are many lumisections, it is technically more effificient to create a list of these dataDicts and concat those all at once)
            if verbose>1: print("DATA DICT:", dataDict)
            dataFrame = pd.Series(dataDict).to_frame().T
            #print(dataFrame)
            detailed_anomaly_df = pd.concat([detailed_anomaly_df, dataFrame])
            #continue to the next loop
            continue

        #If there is more than one anomalous powergroup in that lumisection and it's NOT the whole quarter out
        all_powergroup_combos = itertools.combinations(powergroups, 2)
        #Loop over all pairs of powergroups and search for multi-disk anomalies
        dataDictList = [] #Create a list to store all of the possible Single Disk anomalies in case there are multiple anomalies but no Multi Disk anomaly
        for powergroup_combo in all_powergroup_combos:
            #print("ADFJNDKFN", powergroup_combo)
            anomaly_type = powerGroupsToAnomalyType(powergroup_combo[0], powergroup_combo[1])
            #Extract relevant information
            m_or_p_one, I_or_O_one, disk_number_one, part_number_one = analyzePowerGroupString(powergroup_combo[0])
            m_or_p_two, I_or_O_two, disk_number_two, part_number_two = analyzePowerGroupString(powergroup_combo[1])
            if anomaly_type == "Multi-Disk":
                #Fill in the data dict
                dataDict["Powergroup"] = powergroup_combo[0] + ':' + powergroup_combo[1]
                dataDict["Disk"] = str(disk_number_one) + ':' + str(disk_number_two)
                dataDict["Anomaly_Type"] = "Multi-Disk"
                #Convert the data dict to a pandas dataframe and concat it to the detailed data frame 
                #(NOTE: If there are many lumisections, it is technically more effificient to create a list of these dataDicts and concat those all at once)
                if verbose>1: print("DATA DICT:", dataDict)
                dataFrame = pd.Series(dataDict).to_frame().T
                #print(dataFrame)
                detailed_anomaly_df = pd.concat([detailed_anomaly_df, dataFrame])
                #break
            else:
                #Fill in a data dict for each anomaly
                dataDict["Powergroup"] = powergroup_combo[0]
                dataDict["Disk"] = str(disk_number_one)
                dataDict["Anomaly_Type"] = "Single-Disk"
                #Convert the data dict to a pandas dataframe and concat it to the detailed data frame 
                #(NOTE: If there are many lumisections, it is technically more effificient to create a list of these dataDicts and concat those all at once)
                if verbose>1: print("DATA DICT ONE:", dataDict)
                dataFrame = pd.Series(dataDict).to_frame().T
                if verbose>1: print(dataFrame)
                detailed_anomaly_df = pd.concat([detailed_anomaly_df, dataFrame])

                #Fill in second data dict
                dataDict["Powergroup"] = powergroup_combo[1]
                dataDict["Disk"] = str(disk_number_two)
                dataDict["Anomaly_Type"] = "Single-Disk"
                #Convert the data dict to a pandas dataframe and concat it to the detailed data frame 
                #(NOTE: If there are many lumisections, it is technically more effificient to create a list of these dataDicts and concat those all at once)
                if verbose>1: print("DATA DICT TWO:", dataDict)
                dataFrame = pd.Series(dataDict).to_frame().T
                #print(dataFrame)
                detailed_anomaly_df = pd.concat([detailed_anomaly_df, dataFrame])
            if verbose>0: print('----------------------------------------')
        if verbose>0: print('-----------------------------------------')

    #Remove all exact duplicate rows
    detailed_anomaly_df = detailed_anomaly_df.drop_duplicates()
    if verbose>0: print('\n\n')
    #print(detailed_anomaly_df)
    all_detailed_anomaly_df = pd.concat([all_detailed_anomaly_df, detailed_anomaly_df])
    #ensure the directory is created
    detailed_anomaly_df.to_excel(f"{DIR_NAME}/Raw_PXRing_{RING}_{YEAR}{ERA}_Run_{run_number}.xlsx", index=False, engine='openpyxl')

CPU times: user 187 ms, sys: 33.2 ms, total: 220 ms
Wall time: 779 ms


In [15]:
%%time
# Identify and condense runs of consecutive lumisections for each powergroup
condensed_df = condense_lumisection_runs(all_detailed_anomaly_df)
# print("Condensed consecutive lumisection runs per powergroup:")
# condensed_df = condensed_df.sort_values(by='Start_LS')
# print(condensed_df)
print("Done Condensing Lumisection Runs!")

# Identify and condense overlaping powergroups for each lumisection
condensed_df_again = condense_powergroup_overlap(condensed_df, verbose=False)
# print("Condensed consecutive lumisection runs per Anomaly Type:")
# condensed_df_again = condensed_df_again.sort_values(by='Start_LS')

# Rename columns
renamed_df = condensed_df_again.rename(
    columns={
        "Run_Number": "Run",
        "Start_LS": "First LS",
        "End_LS": "Last LS",
        "Num_LS": "Total LS",
        "Powergroup": "Barrel/Forward",   # will overwrite below
        "Disk": "Layer/Disk",
        "Ring_Num": "Row/Quarter",
        "Anomaly_Type": "Type",
    }
)

# Force Barrel/Forward to "Forward" for all rows
renamed_df["Barrel/Forward"] = "Forward"

# Add "Disk " before the disk number (keep sign)
renamed_df["Layer/Disk"] = renamed_df["Layer/Disk"].apply(lambda x: f"Disk {x}")

# Add "Ring " before the ring number
renamed_df["Row/Quarter"] = renamed_df["Row/Quarter"].apply(lambda x: f"Ring {x}")

# Save to Excel
excel_file = f"Anomalies_PXRing_{RING}_{YEAR}{ERA}_v{VERSION}_period_{PERIOD}_edition_{EDITION}.xlsx"
excel_file_path = f"{DIR_NAME}/../{excel_file}"
renamed_df.to_excel(excel_file_path, index=False, engine='openpyxl')
print(f"Final Anomaly Excel File Saved at `{excel_file}`")

Done Condensing Lumisection Runs!
Final Anomaly Excel File Saved at `Anomalies_PXRing_1_2024C_v1_period_1_edition_1.xlsx`
CPU times: user 29 ms, sys: 5.52 ms, total: 34.5 ms
Wall time: 106 ms


# Extra <---------------

# 7. Optional (Save plot of each anomalous lumisection)

In [13]:
%%time

doPlotting = DO_PLOTTING
saveFigs = SAVE_FIGS
showFigs = SHOW_FIGS
verbose = 1
imageDir = f"images/{DIR_NAME}"
if saveFigs: 
    #Ensure the saving directory exists
    if not os.path.exists(f"images/{DIR_NAME}"): os.makedirs(f"images/{DIR_NAME}")
    print(f"Images will be saved in: {imageDir}")
#Create lists for tracking the anomalous lumisections in all of the runs
# all_anomalous_runs
# all_anomalous_lumisections
# all_anomalous_lumisections_unique
# all_anomalous_powergroups
# total_anomalous_lumisections = len(all_anomalous_lumisections_unique) Either one works
total_anomalous_lumisections = num_anomalous_lumisections
#Inspect the original data, the prediction, the loss, and the binary loss
print(f"There are {total_anomalous_lumisections} anomalous lumisections from run {all_runs[0]} to run {all_runs[-1]}!")
#print(f"The anomalous powergroups are:\n{anomalous_powergroups}\n")
counter = 0 #Counter for keeping track of if we're almost done
previous_counter = 0 #Just used to make the print statements cleaner. Only print progress if the data_dict actually had an anomalous lumisection
for i, data_dict in enumerate(data_dict_list):
    startTime = time.time()
    #Extract the relevant information from the data_dict
    run_number = data_dict['run_number']
    lumisections = data_dict['lumisections']
    multi_lumi_data = data_dict['data_cross']
    mes_pred_cross = data_dict['predictions_cross']
    losses_cross = data_dict['losses']
    losses_binary_cross = data_dict['losses_binary']
    anomalous_lumisections = data_dict['anomalous_lumisections']
    anomalous_powergroups = data_dict['anomalous_powergroups']
    
    previous_lumisection = -1
    for j, lumisection in enumerate(anomalous_lumisections):
        counter += 1 #Increase the counter for each anomalous lumisection plotted
        index = lumiToIndex(lumisections, lumisection)
        data = multi_lumi_data[index]
        prediction = mes_pred_cross[index]
        plot_losses = losses_cross[index]
        plot_losses_binary = losses_binary_cross[index]
        anomalous_powergroup = anomalous_powergroups[j]

        if verbose>1: print(f"Anomalous Power Group: {anomalous_powergroup} \t in Lumisection: {lumisection} \t in Run: {run_number}")
        #Plotting and determining whether the current lumisection is the same as the previous one due to multi disk anomalies
        if doPlotting and lumisection != previous_lumisection: #only print if the current lumisection is not the previous one again
            testingFig, testingAxes = plot_testing_plots(data, prediction, plot_losses, plot_losses_binary, run_number, lumisection, ring_num, 
                                                         directory=imageDir, saveFig=saveFigs, showFig=showFigs)
            #plt.show()
            if verbose>0 and showFigs: 
                print(f"The Anomalous Powergroup in the Above Image is: {anomalous_powergroup}")
        elif doPlotting and lumisection == previous_lumisection:
            if verbose>0 and showFigs: print(f"Due to a Multi-Disk Anomaly, the Other Anomalous Powergroup is: {anomalous_powergroup}")
        
        previous_lumisection = lumisection
    #If verbose and there was an anomalous lumisection in that run
    if verbose>0 and counter != previous_counter: print(f"Done Run {run_number} and {counter}/{total_anomalous_lumisections} Anomalous Lumisections in {time.time()-startTime} Seconds!")
    previous_counter = counter

NameError: name 'SHOW_FIGS' is not defined

## Extra: Plot Specific Data, Predictions, Losses, and Binary Losses

In [ ]:
%%time
#Inspect the original data, the prediction, the loss, and the binary loss
run_number = 393240
lumi_number = 3083
verbose = 1
save_fig = False

#Find the data_dict with a run number of run_number and extract the relevant information
#https://stackoverflow.com/questions/8653516/search-a-list-of-dictionaries-in-python. See here
plot_dict = next((data_dict for data_dict in data_dict_list if data_dict["run_number"] == run_number), None)
if plot_dict == None:
    raise Exception(f"Run Number {run_number} is not in the list of dictionaries!")

#Extract the relavant information for plotting
lumisections = plot_dict['lumisections']
if verbose>1: print(f"There are {len(lumisections)} Available Lumisections:\n{lumisections}")
multi_lumi_data = plot_dict['data_cross']
mes_pred_cross = plot_dict['predictions_cross']
losses_cross = plot_dict['losses']
losses_binary_cross = plot_dict['losses_binary']

#Extract the anomalous powergroups in that lumisection
anomalous_lumisections = plot_dict['anomalous_lumisections'] #anomalous_lumisections is a comprehensive list of anomalous lumisections, 
#Containing duplicates for lumisections with multiple anomalous powergroups
desired_lumi_indices = np.where(anomalous_lumisections == lumi_number)[0] #np.where needs this extra [0]
anomalous_powergroups = plot_dict['anomalous_powergroups']
desired_anomalous_powergroups = anomalous_powergroups[desired_lumi_indices]


#Extract the specific lumisection data
index = lumiToIndex(lumisections, lumi_number)
data = multi_lumi_data[index]
prediction = mes_pred_cross[index]
plot_losses = losses_cross[index]
plot_losses_binary = losses_binary_cross[index]
if verbose>0: print(f"All Anomalous Powergroups in This Lumisection: {desired_anomalous_powergroups}")


print(f"Run Number: {run_number}\tLumi Number: {lumi_number}")
testingFig, testingAxes = plot_testing_plots(data, prediction, plot_losses, 
                                             plot_losses_binary, run_number, lumi_number, ring_num, 
                                             saveFig=save_fig, showFig=True)

plt.show()

## Extra: Inspect Specific Data Dictionaries

In [ ]:
%%skip True
run_number = 393240
verbose = 1
show_fig = True
save_fig = True
save_directory = 'images'

inspect_dict = next((data_dict for data_dict in data_dict_list if data_dict["run_number"] == run_number), None)

print(inspect_dict["run_number"])
print(inspect_dict["lumisections"])
print(inspect_dict["data"].shape)

#Extract the relavant information for plotting
lumisections = inspect_dict['lumisections']
if verbose>1: print(f"There are {len(lumisections)} Available Lumisections:\n{lumisections}")
multi_lumi_data = inspect_dict['data_cross']
mes_pred_cross = inspect_dict['predictions_cross']
losses_cross = inspect_dict['losses']
losses_binary_cross = inspect_dict['losses_binary']

#Extract the anomalous powergroups in that lumisection
anomalous_lumisections = inspect_dict['anomalous_lumisections'] #anomalous_lumisections is a comprehensive list of anomalous lumisections, 
#Containing duplicates for lumisections with multiple anomalous powergroups

#Save all of the images for a specific length of lumisections and for a specific run for analyzing results more in-depth
#Useful for finding anomalies in training data that may not have been found
lumi_start = 3083
lumi_end = 3494
lumi_range = np.arange(lumi_start, lumi_end+1, 1) #Set up the range of lumis to plot
#Exclude lumis where we already know there is an anomaly and they have already been discarded from training
excluded_lumis = np.array([168, 169, 523, 524, 525, 526, 573, 574, 575, 576, 577, 578, 853, 854, 855, 856, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224 ,225 ,226 ,227 ,228 ,229 ,230 ,231 ,232 ,233 ,234 ,235 ,236 ,237 ,238 ])
# lumis_to_plot = [i for i in lumi_range if i not in excluded_lumis] #list comprehension way
#Numpy way
indices=np.argwhere(np.isin(lumi_range,excluded_lumis))
lumis_to_plot=np.delete(lumi_range,indices)
print(lumis_to_plot.shape)

for idx, lumisection in enumerate(lumis_to_plot):
    
    #Only plot every tenth lumisection
    if lumisection % 10 != 0:
        continue

    desired_lumi_indices = np.where(anomalous_lumisections == lumisection)[0] #np.where needs this extra [0]
    anomalous_powergroups = inspect_dict['anomalous_powergroups']
    desired_anomalous_powergroups = anomalous_powergroups[desired_lumi_indices]


    #Extract the specific lumisection data
    try:
        index = lumiToIndex(lumisections, lumisection)
    except:
        print(f"ERROR: Desired Lumisection {lumisection} not found in lumisection array!")
        continue
    data = multi_lumi_data[index]
    prediction = mes_pred_cross[index]
    plot_losses = losses_cross[index]
    plot_losses_binary = losses_binary_cross[index]
    if verbose>0: print(f"All Anomalous Powergroups in This Lumisection: {desired_anomalous_powergroups}")


    print(f"Run Number: {run_number}\tLumi Number: {lumisection}")
    testingFig, testingAxes = plot_testing_plots(data, prediction, plot_losses, 
                                                 plot_losses_binary, run_number, lumisection, ring_num, 
                                                 saveFig=save_fig, showFig=show_fig, directory=save_directory)

    plt.show()

In [7]:
%%time
import numpy as np

chunk_size = 2000  # tune depending on memory limits

for data_dict in data_dict_list:
    data = data_dict["data"].astype(np.float32, copy=False)
    preds = data_dict["predictions"].astype(np.float32, copy=False)

    # Preallocate output arrays once (same shape as input)
    losses = np.empty_like(data, dtype=np.float32)
    losses_binary = np.empty_like(data, dtype=bool)

    # Process in chunks along the first axis
    for i in range(0, data.shape[0], chunk_size):
        sl = slice(i, i + chunk_size)

        # In-place diff and square
        diff = data[sl] - preds[sl]
        np.square(diff, out=diff)

        # Save results into the preallocated arrays
        losses[sl] = diff
        losses_binary[sl] = diff > LOSS_THRESHOLD

    # Add the crosses back (try to make add_cross in-place if possible)
    losses_cross = add_cross(losses)
    losses_binary_cross = add_cross(losses_binary)

    # Store results
    data_dict["losses"] = losses_cross
    data_dict["losses_binary"] = losses_binary_cross


CPU times: user 5.75 s, sys: 833 ms, total: 6.59 s
Wall time: 6.6 s
